In [1]:
import h5py
import os
import subprocess
import tensorflow as tf

''

## Precursors

In [2]:
if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)
    


    

In [3]:
# if not os.path.isdir('models/heart'):
#     os.mkdir('models/heart')
# if not os.path.isfile('models/heart/model_best.tf.meta'):
#     subprocess.call('curl -o models/heart/model_best.tf.index https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.index', shell=True)
#     subprocess.call('curl -o models/heart/model_best.tf.meta https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.meta', shell=True)
#     subprocess.call('curl -o models/heart/model_best.tf.data-00000-of-00001 https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.data-00000-of-00001', shell=True)
    
    
# if not os.path.isdir('models/heart'):
#     os.mkdir('models/heart')
# if not os.path.isfile('models/heart/model_best.h5'):
#     subprocess.call('curl -o models/heart/model_best.h5 https://storage.googleapis.com/basenji_barnyard/model_human.h5', shell=True)
#     subprocess.call('curl -o models/heart/model_best.tf https://storage.googleapis.com/basenji_barnyard/model_human.tf', shell=True)

if not os.path.isdir('models/lcl'):
    os.mkdir('models/lcl')
if not os.path.isfile('models/lcl/model_human.tf.meta'):
#     subprocess.call('curl -o models/lcl/model_human.h5 https://storage.googleapis.com/basenji_barnyard/model_human.h5', shell=True)
    subprocess.call('curl -o models/lcl/model_human.tf.index https://storage.googleapis.com/basenji_barnyard/tf1/model_human.tf.index', shell=True)
    subprocess.call('curl -o models/lcl/model_human.tf.meta https://storage.googleapis.com/basenji_barnyard/tf1/model_human.tf.meta', shell=True)
    subprocess.call('curl -o models/lcl/model_human.tf.data-00000-of-00001 https://storage.googleapis.com/basenji_barnyard/tf1/model_human.tf.data-00000-of-00001', shell=True)
    

In [4]:
lines = [['index','identifier','file','clip','sum_stat','description']]
# lines.append(['0', 'CNhs11760', 'data/CNhs11760.bw', '384', 'sum', 'aorta'])
# lines.append(['1', 'CNhs12843', 'data/CNhs12843.bw', '384', 'sum', 'artery'])
# lines.append(['2', 'CNhs12856', 'data/CNhs12856.bw', '384', 'sum', 'pulmonic_valve'])
lines.append(['0', 'ENCFF093VXI', 'data/ENCFF093VXI.w5', '32', 'sum', 'DNASE:GM12878'])

samples_out = open('data/lcl_wigs.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

## SNP activity difference compute

Analyzing noncoding variation associated with disease is a major application of Basenji. I now offer several tools to enable that analysis. If you have a small set of variants and know what datasets are most relevant, [basenji_sat_vcf.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_vcf.py) lets you perform a saturation mutagenesis of the variant and surrounding region to see the relevant nearby motifs.

If you want scores measuring the influence of those variants on all datasets,
 * [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) computes my SNP activity difference (SAD) score--the predicted change in aligned fragments to the region.
 * [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py) computes my SNP expression difference (SED) score--the predicted change in aligned fragments to gene TSS's.

Here, I'll demonstrate those two programs. You'll need
 * Trained model
 * Input file (FASTA or HDF5 with test_in/test_out)

First, you can either train your own model in the [Train/test tutorial](https://github.com/calico/basenji/blob/master/tutorials/train_test.ipynb) or use one that I pre-trained from the models subdirectory.

As an example, we'll study a prostate cancer susceptibility allele of rs339331 that increases RFX6 expression by modulating HOXB13 chromatin binding (http://www.nature.com/ng/journal/v46/n2/full/ng.2862.html).

First, we'll use [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) to predict across the region for each allele and compute stats about the mean and max differences.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| --cpu | True | Run on CPU (and avoid a GPU prefetch op.) |
| -f | data/hg19.ml.fa | Genome fasta. |
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| --h5 | True | Write output to HDF5. |
| -o | rfx6_sad | Outplot plot directory. |
| --rc | True | Ensemble predictions for forward and reverse complement sequences. |
| --shift | 1,0,-1 | Ensemble predictions for sequences shifted by 1, 0, and -1 bp. |
| -t | data/heart_wigs.txt | Target labels. |
| params_file | models/params_small.txt | Table of parameters to setup the model architecture and optimization parameters. |
| model_file | models/heart/model_best.tf | Trained saved model prefix. |
| vcf_file | data/rs339331.vcf | VCF file specifying variants to score. |

In [7]:
! basenji_sad.py --cpu -f data/hg19.ml.fa -o output/rfx6_sad --rc --shift "1,0,-1" -t data/lcl_wigs.txt models/params.txt models/lcl/model_human.tf data/dsQTL_small.eval.vcf

# ! ../bin/basenji_sad.py -f data/hg19.ml.fa -o output/rfx6_sad --rc --shift "1,0,-1" -t data/heart_wigs.txt models/params_small.txt models/heart/model_best.tf data/rs339331.vcf



{'seq_length': 131072, 'target_pool': 128, 'target_length': 1024, 'num_genomes': 2, 'num_targets': [5313, 1643], 'batch_size': 4, 'batch_buffer': 8192, 'nonlinearity': 'gelu', 'link': 'softplus', 'loss': 'poisson', 'optimizer': 'sgd', 'learning_rate': 0.15, 'momentum': 0.99, 'grad_clip': 2, 'learning_decay_steps': 3000000, 'learning_decay_rate': 0.5, 'batch_norm_momentum': 0.9, 'cnn_l2_scale': 1e-09, 'final_l1_scale': 1e-09, 'architecture': 'dilated_residual', 'conv_dna_filters': 288, 'conv_dna_filter_size': 15, 'conv_dna_pool': 2, 'conv_dna_dropout': 0, 'conv_reduce_pool': 2, 'conv_reduce_dropout': 0, 'conv_dilate_filters': 384, 'conv_dilate_rate_mult': 1.5, 'conv_dilate_rate_max': 96, 'conv_dilate_dropout': 0.3, 'conv_final_filters': 1536, 'conv_final_dropout': 0.05, 'conv_reduce_filter_size': 5, 'conv_reduce_filters_mult': 1.1776}

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your i

## SNP activity difference output

The output HDF5 stores the SNP and target information and predicted scores.

In [9]:
sad_h5 = h5py.File('output/rfx6_sad/sad.h5', 'r')
# list(sad_h5.keys())

In [10]:
list(sad_h5['target_labels'])

[b'DNASE:GM12878']

In [11]:
for snp_key in ['snp', 'chr', 'pos', 'ref']:
    print(snp_key, sad_h5[snp_key][:])

snp [b'rs141671872' b'rs7417106' b'rs6603785' b'rs112571439' b'rs9661285'
 b'rs55665824' b'rs11121820' b'rs2236055' b'rs9919124' b'rs2816057'
 b'rs74056743' b'rs79931859' b'rs1994859' b'rs4147103' b'rs12079905'
 b'rs34958581' b'rs3768324' b'rs12118033' b'rs3122128' b'rs56265163'
 b'rs662145' b'rs10888907' b'rs593257' b'rs6701583' b'rs11207905'
 b'rs11589629' b'rs599134' b'rs17032139' b'rs640195' b'rs2798892'
 b'rs1409367' b'rs999418' b'rs12057448' b'rs6656494' b'rs75500098'
 b'rs60205880' b'rs12064771' b'rs10911363' b'rs789185' b'rs4511180'
 b'rs11119901' b'rs1152837' b'rs75714319' b'rs12741252' b'rs3738398'
 b'rs4658633' b'rs4073056' b'rs59868117' b'rs10794743' b'rs34174118'
 b'rs11819695' b'rs10905307' b'rs141802172' b'rs17152034' b'rs10796127'
 b'rs2793109' b'rs2377981' b'rs59082403' b'rs12269414' b'rs10823321'
 b'rs11000015' b'rs10824083' b'rs10824084' b'rs77570953' b'rs4934157'
 b'rs10887562' b'rs7905184' b'rs7477274' b'rs59063780' b'rs7094325'
 b'rs12265653' b'rs73351021' b'rs705

In [12]:
for ti in range(1):
    for j in range(100):
        cols = (ti, sad_h5['pos'][j], sad_h5['chr'][j], sad_h5['SAD'][j,ti], sad_h5['target_ids'][ti], sad_h5['target_labels'][ti])
        print('%2d %s %s %7.4f  %12s  %s' % cols)

 0 856583 b'chr1'  0.6777  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 911595 b'chr1' -1.6309  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 1186502 b'chr1' -0.1852  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 1227412 b'chr1' -8.7266  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 1590575 b'chr1' -7.5000  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 3369847 b'chr1' -1.5635  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 11809402 b'chr1' -0.8521  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 12042261 b'chr1'  0.0206  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 15711053 b'chr1' -0.8457  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 18902257 b'chr1'  2.0449  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 19397741 b'chr1'  0.2798  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 25876340 b'chr1' -0.0476  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 29497820 b'chr1'  0.1530  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 31255905 b'chr1' -1.7344  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 32059773 b'chr1' -3.6562  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 32936807 b'chr1'  1.7119  b'ENCFF093VXI'  b'DNASE:GM12878'
 0 39492462 b'ch

ValueError: Index (99) out of range (0-98)

These are inconclusive small effect sizes, not surprising given that we're only studying heart CAGE. The proper cell types and experiments would shed more light.

## SNP expression difference compute

Alternatively, we can directly query the predictions at gene TSS's using [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py). Note that I haven't revised these scripts to make use of tf.data, so they are a bit less wieldy right now.

[basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py) takes as input the gene sequence HDF5 format described in [genes.ipynb](https://github.com/calico/basenji/blob/master/tutorials/genes.ipynb). There's no harm to providing an HDF5 that describes all genes, but it's too big to easily move around so I constructed one that focuses on RFX6.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| -o | rfx6_sed | Outplot plot directory. |
| --rc | | Predict forward and reverse complement versions and average the results. |
| -w | 128 | Sequence bin width at which predictions are made. |
| params_file | models/params_med.txt | Table of parameters to setup the model architecture and optimization parameters. |
| model_file | models/gm12878.tf | Trained saved model prefix. |
| genes_hdf5_file | data/rfx6.h5 | HDF5 file specifying gene sequences to query. |
| vcf_file | data/rs339331.vcf | VCF file specifying variants to score. |

Before running [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py), we need to generate an input data file for RFX6. Using an included GTF file that contains only RFX6, one can use [basenji_hdf5_genes.py](https://github.com/calico/basenji/blob/master/bin/basenji_hdf5_genes.py) to create the required format.

In [25]:
! ../bin/basenji_hdf5_genes.py -g data/human.hg19.genome -l 131072 -c 0.333 -w 128 data/hg19.ml.fa data/rfx6.gtf data/rfx6.h5



In [26]:
! ../bin/basenji_sed.py -a -g data/human.hg19.genome -o output/rfx6_sed --rc models/params.txt models/lcl/model_human.tf data/rfx6.h5 data/rs339331.vcf



/Users/andyjiang/opt/anaconda3/envs/basenji_old/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/andyjiang/opt/anaconda3/envs/basenji_old/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/andyjiang/opt/anaconda3/envs/basenji_old/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

## SNP expression difference compute

In [23]:
! sort -k9 -g output/rfx6_sed/sed_gene.txt

rsid ref alt gene tss_dist ref_pred alt_pred sed ser target_index target_id target_label


In [24]:
! sort -k9 -gr output/rfx6_sed/sed_gene.txt

rsid ref alt gene tss_dist ref_pred alt_pred sed ser target_index target_id target_label
